# LightGBM v1

### Imports

In [1]:
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

### Funciones

In [2]:
def normalizar_texto(descripcion):
    
    descripcion = descripcion.replace(',', ' ')
    descripcion = descripcion.replace('.', ' ')
    descripcion = descripcion.replace('<', ' ')
    descripcion = descripcion.replace('>', ' ')
    descripcion = descripcion.replace('\n', ' ')
    descripcion = descripcion.replace('\strong', ' ')
    descripcion = descripcion.replace('\bold', ' ')
    descripcion = descripcion.lower()
    
    return descripcion

def contar_palabras(titulo, palabras):
    
    contadas = 0
    
    for palabra in palabras:
        spliteado = titulo.split()
        contadas += spliteado.count(palabra)
        
    return contadas

### Reads
train.csv -> train
<br>
test.csv -> test

In [6]:
train = pd.read_csv('./train.csv')
#test = pd.read_csv('../data/test.csv')

### Preparo el modelo 

**Feature Direción:**

In [7]:
DIRINVALIDAS = set(['Calle','domicilio conocido','calle','CALLE','--','Sin nombre ','1','.','-','x #x','0','...','S/N'])

df = train

df.loc[df.direccion.isin(DIRINVALIDAS), 'direccion'] = 0
df.loc[df['direccion'] != 0, 'direccion'] = 1

df['direccion'].value_counts()

1    237759
0      2241
Name: direccion, dtype: int64

**Feature Titulo y Descripcion:**

In [8]:
df['titulo'] = df['titulo'].fillna(value = ".")
df['titulo'] = df['titulo'].apply(normalizar_texto)
df['descripcion'] = df['descripcion'].fillna(value = ".")
df['descripcion'] = df['descripcion'].apply(normalizar_texto)

df['calefaccion'] = df['titulo'].apply(contar_palabras, palabras=["calefaccion", "calefacción", "calefaccionado", "aireacondicinado",
                    "acondicionado", "estufa","chimenea"]) + df['descripcion'].apply(contar_palabras, palabras=["calefaccion", 
                    "calefacción", "calefaccionado", "aireacondicinado", "acondicionado", "estufa","chimenea"])

df['suite'] = df['titulo'].apply(contar_palabras, palabras=["suite"])+df['descripcion'].apply(contar_palabras, palabras=["suite"])

df['avenida'] = df['titulo'].apply(contar_palabras, palabras=["avenida", "av", "avenidas", "bulevar", 
               "boulevard", "paseo", "vía"])+df['descripcion'].apply(contar_palabras, palabras=["avenida", "av", 
                "avenidas", "bulevar","boulevard", "paseo", "vía"])

df['gim'] = df['titulo'].apply(contar_palabras, palabras=["gimnasio", "gimnásio", "entrenamiento", "gim", "gym", "fit",
            "ejercicio", "gimnasia","atletismo", "cancha"])+df['descripcion'].apply(contar_palabras, palabras=["gimnasio",
            "gimnásio", "entrenamiento", "gim", "gym", "fit","ejercicio", "gimnasia","atletismo", "cancha"])

df['cochera'] = df['titulo'].apply(contar_palabras, palabras=["cochera", "cocheras", "garage", 
                "garages", "garaje", "garajes"])+df['descripcion'].apply(contar_palabras, palabras=["cochera", 
                "cocheras", "garage","garages", "garaje", "garajes"])

df['ubicacion'] = df['titulo'].apply(contar_palabras, palabras=["ubicacion", "ubicación", "locacion", 
                "locación", "localizacion", "localización","ubicado", "ubicada", "centro", "centrico",
                "centrica", "céntrico", "céntrica","central"])+df['descripcion'].apply(contar_palabras, palabras=["ubicacion", "ubicación", "locacion", 
                "locación", "localizacion", "localización","ubicado", "ubicada", "centro", "centrico",
                "centrica", "céntrico", "céntrica","central"])

df['balcon'] = df['titulo'].apply(contar_palabras, palabras=["balcon", "balcón", "terraza", "palco", "mirador", 
            "balconcillo","azotea", "solana"])+df['descripcion'].apply(contar_palabras, palabras=["balcon", "balcón", 
            "terraza", "palco", "mirador", "balconcillo","azotea", "solana"])

df['camaraseg'] = df['titulo'].apply(contar_palabras, palabras=["camara", "cámara", "cámaras", "camaras","seguridad",
                "guardia","seguro","protegido"]) + df['descripcion'].apply(contar_palabras, palabras=["camara", "cámara",
                "cámaras", "camaras","seguridad","guardia","seguro","protegido"])

df['parque'] = df['titulo'].apply(contar_palabras, palabras=["parque", "plaza", "plazoleta", "glorieta","jardin",
            "jardín","patio"])+df['descripcion'].apply(contar_palabras, palabras=["parque", "plaza", "plazoleta", 
            "glorieta","jardin","jardín","patio"])

df['amoblado'] = df['titulo'].apply(contar_palabras, palabras=["muebles", "amoblado", "mueble","decorado",
                "listo"]) + df['descripcion'].apply(contar_palabras, palabras=["muebles", "amoblado", "mueble",
                "decorado","listo"])

df['bañera'] = df['titulo'].apply(contar_palabras, palabras=["bañera", "hidromasaje", "hidro", "tina",
            "jacuzzi","jacuzi","yacuzi"]) + df['descripcion'].apply(contar_palabras, palabras=["bañera", "hidromasaje", 
            "hidro", "tina","jacuzzi","jacuzi","yacuzi"])


df['estreno'] = df['titulo'].apply(contar_palabras, palabras=["nuevo", "nueva", "estrenar","estreno",
            "innovador"]) + df['descripcion'].apply(contar_palabras, palabras=["nuevo", "nueva", "estrenar","estreno",
            "innovador"])

df['transporte'] = df['titulo'].apply(contar_palabras, palabras=["subte", "subterraneo", "subterráneo","metro", 
                "estacion", "estación", "tren","subestacion", "subestación", "ferrocarril","metrobús", "metrobus", 
                "trolebus","trolebús", "bus", "bús"]) + df['descripcion'].apply(contar_palabras, palabras=["subte", 
                "subterraneo", "subterráneo","metro", "estacion", "estación", "tren","subestacion", "subestación", 
                "ferrocarril","metrobús", "metrobus", "trolebus","trolebús", "bus", "bús"])

df['pileta'] = df['titulo'].apply(contar_palabras, palabras=["piscina", "pileta", "nado"])+ df['descripcion'].apply(contar_palabras, palabras=["piscina", "pileta", "nado"])
                                       
df['lujo'] = df['titulo'].apply(contar_palabras, palabras=["lujo", "delujo", "deluxe", "delúxe", "lujosa", "lujoso", 
            "lujosas", "lujosos","exclusivo","vip"]) + df['descripcion'].apply(contar_palabras, palabras=["lujo", 
            "delujo","deluxe", "delúxe", "lujosa", "lujoso", "lujosas", "lujosos","exclusivo","vip"])

df['humilde'] = df['titulo'].apply(contar_palabras, palabras=["humilde", "economico", "economica", 
            "económico", "económica", "barata", "barato", "accesible", "baratillo",
            "baratilla", "rebajado", "ganga", "asequible", "módico", "módica","credito","crédito","oferta","oferton",
            "imperdible"]) + df['descripcion'].apply(contar_palabras, palabras=["humilde", "economico", "economica", 
            "económico", "económica", "barata", "barato", "accesible", "baratillo",
            "baratilla", "rebajado", "ganga", "asequible", "módico", "módica",
            "credito","crédito","oferta","oferton","imperdible"]) 

df['ventana'] = df['titulo'].apply(contar_palabras, palabras=["ventana", "ventanas", 
            "vista", "ventanal","vistas","cristal"]) + df['descripcion'].apply(contar_palabras, palabras=["ventana",
            "ventanas", "vista", "ventanal","vistas","cristal"])

df['nuevo'] = df['titulo'].apply(contar_palabras, palabras=["reciente", "recien", "recién", "nueva", "nuevo", "nuevas", 
            "nuevos", "estrenar"]) + df['descripcion'].apply(contar_palabras, palabras=["reciente", "recien", "recién", 
            "nueva", "nuevo", "nuevas", "nuevos", "estrenar"])

df['luz'] = df['titulo'].apply(contar_palabras, palabras=["luz", "luminoso", "luminosa","claridad", "luminiscencia", 
        "luminosidad", "iluminación","iluminacion"]) + df['descripcion'].apply(contar_palabras, palabras=["luz", 
        "luminoso", "luminosa","claridad", "luminiscencia","luminosidad", "iluminación","iluminacion"])

df['bueno'] = df['titulo'].apply(contar_palabras, palabras=["bueno", "buena", "buenas", "buenos","excelente", 
            "excelentes","increible","espectacular"]) + df['descripcion'].apply(contar_palabras, palabras=["bueno", 
            "buena", "buenas", "buenos","excelente", "excelentes","increible","espectacular"])

df['contable'] = df['titulo'].apply(contar_palabras, palabras=["precio"]) + df['descripcion'].apply(contar_palabras, palabras=["precio"])

#Nuevos Feat

df['agente'] = df['descripcion'].apply(contar_palabras, palabras=["inmobiliaria", "asesoria", "asesoría", "lider", "re/max", "remax"]) \
             + df['titulo'].apply(contar_palabras, palabras=["inmobiliaria", "asesoria", "asesoría", "lider", "re/max", "remax"])

df['garante'] = df['descripcion'].apply(contar_palabras, palabras=["garante", "garantía", "fiador", "garantizador", "avalista", "garantia",
                "defensor", "garantías", "garantes", "codeudor"]) + df['titulo'].apply(contar_palabras, palabras=["garante", 
                "garantía", "fiador", "garantizador", "avalista", "garantia", "defensor", "garantías", "garantes", "codeudor"])

df['finanza'] =  df['descripcion'].apply(contar_palabras, palabras=["credito", "crédito", "prestamo", "préstamo", "cuotas", "pagos", "hipotecario"\
                 ,"amortizable", "aptocredito", "aptocrédito", "apto-credito", "apto-crédito", "aptocredito", "aptocréditp", "ahorro"]) + \
                 df['titulo'].apply(contar_palabras, palabras=["credito", "crédito", "prestamo", "préstamo", "cuotas", "pagos", "hipotecario" \
                 ,"amortizable", "aptocredito", "aptocrédito", "apto-credito", "apto-crédito", "aptocredito", "aptocrédito", "ahorro"]) 

df['turismo'] =  df['descripcion'].apply(contar_palabras, palabras=["playa", "vacaciones", "descanso", "costa", "arena", "mar", "montaña", "monte",
                                                                   "paisaje", "orilla", "rambla", "turista", "turistas", "cordillera", "sierra"])\
                 + df['titulo'].apply(contar_palabras, palabras=["playa", "vacaciones", "descanso", "costa", "arena", "mar", "montaña", "monte",
                                                                   "paisaje", "orilla", "rambla", "turista", "turistas", "cordillera", "sierra"])

df['longitud_desc'] =  df['descripcion'].apply(lambda x: len(x.split()))

#Nuevas de V4

df['longitud_titulo'] = df['titulo'].apply(lambda x: len(x))

df['cant_!'] = df['titulo'].apply(lambda x: x.count('!')) 

df['tranquilidad'] = df['descripcion'].apply(contar_palabras, palabras=["armonía", "armonia", "tranquilo", "tranqui", "tranquilidad", \
                                                                        "paz", "calma", "calmo", "quietud"])\
                 + df['titulo'].apply(contar_palabras, palabras=["armonía", "armonia", "tranquilo", "tranqui", "tranquilidad", \
                                                                        "paz", "calma", "calmo", "quietud"])

df['reparacion'] = df['descripcion'].apply(contar_palabras, palabras=["reparación", "reparacion", "reparando", "reparar", \
                            "construcción", "construccion", "construyendo", "construllendo", "mantenimiento", "averiado", "averiada",
                                                                     "refaccion","refacción","refacciones"])\
                 + df['titulo'].apply(contar_palabras, palabras=["reparación", "reparacion", "reparando", "reparar", \
                            "construcción", "construccion", "construyendo", "construllendo", "mantenimiento", "averiado", "averiada"
                                                                     "refaccion","refacción","refacciones"])

df['mascotas'] = df['descripcion'].apply(contar_palabras, palabras=["mascotas", "mascota", "perros", "perro", "perra", \
                                                "gatos", "gato", "gata", "animal", "animales"])\
                 + df['titulo'].apply(contar_palabras, palabras=["mascotas", "mascota", "perros", "perro", "perra", \
                                                "gatos", "gato", "gata", "animal", "animales"])

df['accesibilidad'] = df['descripcion'].apply(contar_palabras, palabras=["rampa", "discapacitados", "discapacitado", \
                                                "discapacitada", "lisiado", "lisiada", "silla de ruedas", "lastimado", "heridos"])\
                 + df['titulo'].apply(contar_palabras, palabras=["rampa", "discapacitados", "discapacitado", \
                                                "discapacitada", "lisiado", "lisiada", "silla de ruedas", "lastimado", "heridos"])

df['normas'] =  df['descripcion'].apply(contar_palabras, palabras=["norma", "normas", "regla", "reglas", \
                                                "prohibido", "prohibida", "denegado", "denegada", "imposible", \
                                                "ilegal", "legal", "multa", "infraccion", "infracción"])\
                 + df['titulo'].apply(contar_palabras, palabras=["norma", "normas", "regla", "reglas", \
                                                "prohibido", "prohibida", "denegado", "denegada", "imposible", \
                                                "ilegal", "legal", "multa", "infraccion", "infracción"])

df['beneficios'] = df['descripcion'].apply(contar_palabras, palabras=["gratis", "free", "incluido", "incluye", \
                                                "agregado", "gratuito", "gratuitamente", \
                                                "regalo"])\
                 + df['titulo'].apply(contar_palabras, palabras=["gratis", "free", "incluido", "incluye", \
                                                "agregado", "gratuito", "gratuitamente", \
                                                "regalo"])

df['conexion'] = df['descripcion'].apply(contar_palabras, palabras=["wifi", "wi-fi", "internet", "conexión", "conexion", \
                                                                   "señal"])\
                 + df['titulo'].apply(contar_palabras, palabras=["wifi", "wi-fi", "internet", "conexión", "conexion", \
                                                                   "señal"])

df['servicios_desc'] = df['descripcion'].apply(contar_palabras, palabras = ["servicio", "servicios"])

df['metros_desc'] = df['descripcion'].apply(contar_palabras, palabras = ["metros","m2"])

df['acabados'] = df['descripcion'].apply(contar_palabras, palabras = ["acabados","acabado","terminacion","terminación"])

df['plusvalia'] = df['descripcion'].apply(contar_palabras, palabras = ["plusvalia", "plusvalía"])

#Nuevos de V5

df['cocina'] = df['descripcion'].apply(contar_palabras, palabras=['cocina', 'cocinas'])

df['alberca'] = df['descripcion'].apply(contar_palabras, palabras=['alberca'])

df['negacion'] = df['descripcion'].apply(contar_palabras, palabras = ["no"])

df['variospisos'] = df['descripcion'].apply(contar_palabras, palabras = ["escalera","escaleras",
                    "ascensor", "elevador", "escalinata", "gradas", "escalerilla"])

df['vestidor'] = df['descripcion'].apply(contar_palabras, palabras = ["vestidor"])

**Feature Metros:**

In [9]:
df['metrosdescubiertos'] = abs(df['metrostotales']-df['metroscubiertos'])
df['relacionmetros'] = df['metrostotales']/df['metroscubiertos']

In [10]:
def mi_split (df):

    X = df.drop(['precio'], axis = 1)
    Y = df['precio']
    x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.35)
    return x_train, x_test, y_train, y_test

from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn import preprocessing

def minmax(serie):
    
    return (serie-serie.min())/(serie.max()-serie.min())

def mean_target_encoding(train, nombrefeat, nombretarget):
    
    temp = train.groupby(nombrefeat)[nombretarget].transform(np.mean)
    train[nombrefeat + "_mean"]=(temp-temp.min())/(temp.max()-temp.min())
    
    return train

def mean_target_decoding(x_test, nombrefeat, x_train):
    
    nombrefeatmean = nombrefeat + "_mean"
    
    temp = x_train.loc[:,[nombrefeat,nombrefeatmean]]
    temp = temp.set_index(nombrefeat)
    temp = temp.drop_duplicates()
    temp = temp.T.squeeze()
    values = x_test[nombrefeat].map(temp)
    x_test[nombrefeatmean] = values 

    return x_test

def armar_set_2(train):
    
    #Puedo resolver de forma general las que son iguales para train y test
    #es decir, aquellas que no filtran informacion a los de validacion
    
    train = train.drop(['descripcion', 'titulo', 'direccion', 'id'], axis = 1)
    
    #Adiciono servicios
    train['servicios'] = train['piscina'] + train['gimnasio'] + train['usosmultiples']
    
    train['fecha'] = pd.to_datetime(train['fecha']).dt.year
    
    #"Normalizo" la antiguedad
    #train['antiguedad'] = minmax(train['antiguedad'])
    
    #Elimino los residuos
    #train = train.drop(['piscina', 'gimnasio','usosmultiples','escuelascercanas', 
                       # 'centroscomercialescercanos'], axis = 1)
    
    #Hago el split                    
    x_train, x_test, y_train, y_test = mi_split(train)
        
    x_train["precio"] = y_train
    x_test["precio"] = y_test
    
    
    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometro']=df1['precio']/df1['metrostotales']

    PMM = df1.groupby('idzona').agg({'preciometro': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometro": "precioMetroMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')


    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometroCub']=df1['precio']/df1['metroscubiertos']

    PMM = df1.groupby('idzona').agg({'preciometroCub': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroCub": "precioMetroCubMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')
    
    
    
    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometroDes']=df1['precio']/df1['metrosdescubiertos']

    PMM = df1.groupby('idzona').agg({'preciometroDes': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroDes": "precioMetroDesMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')


    
    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometro']=df1['precio']/df1['metrostotales']
    
    PMM = df1.groupby('ciudad').agg({'preciometro': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometro": "precioMetroMeanC"})

    x_train = x_train.merge(PMM, on='ciudad',how='left')
    x_test = x_test.merge(PMM, on='ciudad',how='left')
    
    
    df1 = x_train.dropna(subset=['metroscubiertos'])
    df1['preciometroCub']=df1['precio']/df1['metroscubiertos']

    PMM = df1.groupby('ciudad').agg({'preciometroCub': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroCub": "precioMetroCubMeanC"})

    x_train = x_train.merge(PMM, on='ciudad',how='left')
    x_test = x_test.merge(PMM, on='ciudad',how='left')
    
    
    df1 = x_train.dropna(subset=['metrosdescubiertos'])
    df1['preciometroDes']=df1['precio']/df1['metrosdescubiertos']

    PMM = df1.groupby('ciudad').agg({'preciometroDes': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroDes": "precioMetroDesMeanC"})

    x_train = x_train.merge(PMM, on='ciudad',how='left')
    x_test = x_test.merge(PMM, on='ciudad',how='left')
    
    
    
    #Calculo los mean target
    x_train_mean = mean_target_encoding(x_train, "provincia", "precio") 
    x_train_mean['precio'] = y_train
    x_train_mean = mean_target_encoding(x_train_mean, "tipodepropiedad", "precio")
    x_train_mean = mean_target_encoding(x_train_mean, "ciudad", "precio")
    x_train = mean_target_encoding(x_train_mean, "idzona", "precio")

    
    #Se los asigno a los test (NO LOS CALCULO CON ELLOS!!!!!!)
    x_test = mean_target_decoding(x_test, "provincia", x_train_mean)
    x_test = mean_target_decoding(x_test, "tipodepropiedad", x_train_mean)
    x_test = mean_target_decoding(x_test, "ciudad", x_train_mean)
    x_test = mean_target_decoding(x_test, "idzona", x_train_mean)
    
    backup = x_train_mean
    
    x_train = x_train_mean.drop(["precio","provincia","tipodepropiedad","ciudad"], axis=1)
    x_test.drop(["precio","provincia","tipodepropiedad", "ciudad"], axis=1, inplace = True)

    
    return x_train, x_test, y_train, y_test, backup

In [11]:
x_train, x_test, y_train, y_test, backup = armar_set_2(df)

y = y_train.values

x = x_train.values

y_test = y_test.values

x_test = x_test.values

# divido el df 

/home/belu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/belu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/belu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

### Creo LightGBM data containers

In [12]:
categorical_features = [c for c, col in enumerate(train.columns) if 'cat' in col]
train_data = lightgbm.Dataset(x, label=y, categorical_feature=categorical_features)
test_data = lightgbm.Dataset(x_test, label=y_test)


In [13]:
#Lo entreno

params = {
    'objective': 'regression',
    'colsample_bytree': 0.9731668400523877,
    'learning_rate': 0.05,
    'min_child_samples': 180,#171 
    'min_child_weight': 1e-05, #1e-05
    'num_leaves': 35,#41 
    'reg_alpha': 10, #10
    'reg_lambda': 1000, #100
    'subsample': 0.9,#0.5575732396028996
    'max_depth':-1, 
    'random_state':314, 
    'silent':True,
    'max_bin': 800,
    'metric':'mae', 
    'n_jobs':None, 
    'n_estimators':100000
}

model = lightgbm.train(params,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=10000,
                       early_stopping_rounds=500)

/home/belu/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/belu/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/belu/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1247: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/home/belu/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:794: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


[1]	valid_0's l1: 1.55461e+06
Training until validation scores don't improve for 500 rounds
[2]	valid_0's l1: 1.50526e+06
[3]	valid_0's l1: 1.45817e+06
[4]	valid_0's l1: 1.41316e+06
[5]	valid_0's l1: 1.3706e+06
[6]	valid_0's l1: 1.3303e+06
[7]	valid_0's l1: 1.29245e+06
[8]	valid_0's l1: 1.25684e+06
[9]	valid_0's l1: 1.22321e+06
[10]	valid_0's l1: 1.19193e+06
[11]	valid_0's l1: 1.16261e+06
[12]	valid_0's l1: 1.1345e+06
[13]	valid_0's l1: 1.10742e+06
[14]	valid_0's l1: 1.08273e+06
[15]	valid_0's l1: 1.05854e+06
[16]	valid_0's l1: 1.03569e+06
[17]	valid_0's l1: 1.01494e+06
[18]	valid_0's l1: 994579
[19]	valid_0's l1: 976069
[20]	valid_0's l1: 957856
[21]	valid_0's l1: 940584
[22]	valid_0's l1: 924078
[23]	valid_0's l1: 908717
[24]	valid_0's l1: 894728
[25]	valid_0's l1: 880969
[26]	valid_0's l1: 867083
[27]	valid_0's l1: 854367
[28]	valid_0's l1: 841858
[29]	valid_0's l1: 830397
[30]	valid_0's l1: 819238
[31]	valid_0's l1: 808668
[32]	valid_0's l1: 799030
[33]	valid_0's l1: 789791
[34]	va

[306]	valid_0's l1: 554315
[307]	valid_0's l1: 554236
[308]	valid_0's l1: 554133
[309]	valid_0's l1: 554055
[310]	valid_0's l1: 553989
[311]	valid_0's l1: 553903
[312]	valid_0's l1: 553823
[313]	valid_0's l1: 553715
[314]	valid_0's l1: 553623
[315]	valid_0's l1: 553507
[316]	valid_0's l1: 553391
[317]	valid_0's l1: 553284
[318]	valid_0's l1: 553200
[319]	valid_0's l1: 553131
[320]	valid_0's l1: 553040
[321]	valid_0's l1: 552946
[322]	valid_0's l1: 552872
[323]	valid_0's l1: 552792
[324]	valid_0's l1: 552713
[325]	valid_0's l1: 552619
[326]	valid_0's l1: 552534
[327]	valid_0's l1: 552499
[328]	valid_0's l1: 552403
[329]	valid_0's l1: 552344
[330]	valid_0's l1: 552230
[331]	valid_0's l1: 552151
[332]	valid_0's l1: 552088
[333]	valid_0's l1: 552030
[334]	valid_0's l1: 551965
[335]	valid_0's l1: 551857
[336]	valid_0's l1: 551801
[337]	valid_0's l1: 551678
[338]	valid_0's l1: 551624
[339]	valid_0's l1: 551549
[340]	valid_0's l1: 551487
[341]	valid_0's l1: 551417
[342]	valid_0's l1: 551334
[

[614]	valid_0's l1: 538575
[615]	valid_0's l1: 538561
[616]	valid_0's l1: 538515
[617]	valid_0's l1: 538471
[618]	valid_0's l1: 538451
[619]	valid_0's l1: 538438
[620]	valid_0's l1: 538411
[621]	valid_0's l1: 538380
[622]	valid_0's l1: 538346
[623]	valid_0's l1: 538320
[624]	valid_0's l1: 538278
[625]	valid_0's l1: 538238
[626]	valid_0's l1: 538206
[627]	valid_0's l1: 538156
[628]	valid_0's l1: 538119
[629]	valid_0's l1: 538084
[630]	valid_0's l1: 538026
[631]	valid_0's l1: 538006
[632]	valid_0's l1: 537969
[633]	valid_0's l1: 537934
[634]	valid_0's l1: 537917
[635]	valid_0's l1: 537878
[636]	valid_0's l1: 537859
[637]	valid_0's l1: 537839
[638]	valid_0's l1: 537819
[639]	valid_0's l1: 537805
[640]	valid_0's l1: 537776
[641]	valid_0's l1: 537766
[642]	valid_0's l1: 537726
[643]	valid_0's l1: 537718
[644]	valid_0's l1: 537681
[645]	valid_0's l1: 537647
[646]	valid_0's l1: 537623
[647]	valid_0's l1: 537575
[648]	valid_0's l1: 537548
[649]	valid_0's l1: 537510
[650]	valid_0's l1: 537469
[

[918]	valid_0's l1: 532494
[919]	valid_0's l1: 532486
[920]	valid_0's l1: 532466
[921]	valid_0's l1: 532446
[922]	valid_0's l1: 532444
[923]	valid_0's l1: 532404
[924]	valid_0's l1: 532381
[925]	valid_0's l1: 532366
[926]	valid_0's l1: 532338
[927]	valid_0's l1: 532341
[928]	valid_0's l1: 532338
[929]	valid_0's l1: 532317
[930]	valid_0's l1: 532295
[931]	valid_0's l1: 532265
[932]	valid_0's l1: 532242
[933]	valid_0's l1: 532216
[934]	valid_0's l1: 532198
[935]	valid_0's l1: 532191
[936]	valid_0's l1: 532188
[937]	valid_0's l1: 532174
[938]	valid_0's l1: 532137
[939]	valid_0's l1: 532122
[940]	valid_0's l1: 532104
[941]	valid_0's l1: 532078
[942]	valid_0's l1: 532067
[943]	valid_0's l1: 532044
[944]	valid_0's l1: 532028
[945]	valid_0's l1: 532013
[946]	valid_0's l1: 531995
[947]	valid_0's l1: 531977
[948]	valid_0's l1: 531973
[949]	valid_0's l1: 531961
[950]	valid_0's l1: 531949
[951]	valid_0's l1: 531942
[952]	valid_0's l1: 531924
[953]	valid_0's l1: 531902
[954]	valid_0's l1: 531906
[

...

[13832]	valid_0's l1: 505872
[13833]	valid_0's l1: 505873
[13834]	valid_0's l1: 505873
[13835]	valid_0's l1: 505872
[13836]	valid_0's l1: 505872
[13837]	valid_0's l1: 505873
[13838]	valid_0's l1: 505871
[13839]	valid_0's l1: 505870
[13840]	valid_0's l1: 505870
[13841]	valid_0's l1: 505872
[13842]	valid_0's l1: 505871
[13843]	valid_0's l1: 505873
[13844]	valid_0's l1: 505872
[13845]	valid_0's l1: 505871
[13846]	valid_0's l1: 505870
[13847]	valid_0's l1: 505870
[13848]	valid_0's l1: 505869
[13849]	valid_0's l1: 505868
[13850]	valid_0's l1: 505869
[13851]	valid_0's l1: 505870
[13852]	valid_0's l1: 505873
[13853]	valid_0's l1: 505873
[13854]	valid_0's l1: 505873
[13855]	valid_0's l1: 505874
[13856]	valid_0's l1: 505874
[13857]	valid_0's l1: 505875
[13858]	valid_0's l1: 505875
[13859]	valid_0's l1: 505876
[13860]	valid_0's l1: 505877
[13861]	valid_0's l1: 505876
[13862]	valid_0's l1: 505877
[13863]	valid_0's l1: 505878
[13864]	valid_0's l1: 505880
[13865]	valid_0's l1: 505882
[13866]	valid_

[14115]	valid_0's l1: 505841
[14116]	valid_0's l1: 505840
[14117]	valid_0's l1: 505841
[14118]	valid_0's l1: 505841
[14119]	valid_0's l1: 505843
[14120]	valid_0's l1: 505840
[14121]	valid_0's l1: 505837
[14122]	valid_0's l1: 505835
[14123]	valid_0's l1: 505836
[14124]	valid_0's l1: 505834
[14125]	valid_0's l1: 505831
[14126]	valid_0's l1: 505830
[14127]	valid_0's l1: 505831
[14128]	valid_0's l1: 505830
[14129]	valid_0's l1: 505828
[14130]	valid_0's l1: 505829
[14131]	valid_0's l1: 505830
[14132]	valid_0's l1: 505830
[14133]	valid_0's l1: 505831
[14134]	valid_0's l1: 505832
[14135]	valid_0's l1: 505831
[14136]	valid_0's l1: 505829
[14137]	valid_0's l1: 505831
[14138]	valid_0's l1: 505833
[14139]	valid_0's l1: 505835
[14140]	valid_0's l1: 505839
[14141]	valid_0's l1: 505839
[14142]	valid_0's l1: 505838
[14143]	valid_0's l1: 505837
[14144]	valid_0's l1: 505837
[14145]	valid_0's l1: 505840
[14146]	valid_0's l1: 505838
[14147]	valid_0's l1: 505836
[14148]	valid_0's l1: 505837
[14149]	valid_

[14400]	valid_0's l1: 505767
[14401]	valid_0's l1: 505764
[14402]	valid_0's l1: 505763
[14403]	valid_0's l1: 505763
[14404]	valid_0's l1: 505765
[14405]	valid_0's l1: 505764
[14406]	valid_0's l1: 505761
[14407]	valid_0's l1: 505761
[14408]	valid_0's l1: 505763
[14409]	valid_0's l1: 505765
[14410]	valid_0's l1: 505766
[14411]	valid_0's l1: 505767
[14412]	valid_0's l1: 505767
[14413]	valid_0's l1: 505768
[14414]	valid_0's l1: 505770
[14415]	valid_0's l1: 505770
[14416]	valid_0's l1: 505769
[14417]	valid_0's l1: 505767
[14418]	valid_0's l1: 505766
[14419]	valid_0's l1: 505766
[14420]	valid_0's l1: 505767
[14421]	valid_0's l1: 505768
[14422]	valid_0's l1: 505769
[14423]	valid_0's l1: 505770
[14424]	valid_0's l1: 505768
[14425]	valid_0's l1: 505769
[14426]	valid_0's l1: 505768
[14427]	valid_0's l1: 505768
[14428]	valid_0's l1: 505765
[14429]	valid_0's l1: 505763
[14430]	valid_0's l1: 505760
[14431]	valid_0's l1: 505757
[14432]	valid_0's l1: 505753
[14433]	valid_0's l1: 505753
[14434]	valid_

[14687]	valid_0's l1: 505644
[14688]	valid_0's l1: 505645
[14689]	valid_0's l1: 505640
[14690]	valid_0's l1: 505638
[14691]	valid_0's l1: 505639
[14692]	valid_0's l1: 505640
[14693]	valid_0's l1: 505645
[14694]	valid_0's l1: 505647
[14695]	valid_0's l1: 505648
[14696]	valid_0's l1: 505644
[14697]	valid_0's l1: 505642
[14698]	valid_0's l1: 505642
[14699]	valid_0's l1: 505638
[14700]	valid_0's l1: 505633
[14701]	valid_0's l1: 505632
[14702]	valid_0's l1: 505629
[14703]	valid_0's l1: 505627
[14704]	valid_0's l1: 505628
[14705]	valid_0's l1: 505629
[14706]	valid_0's l1: 505628
[14707]	valid_0's l1: 505629
[14708]	valid_0's l1: 505629
[14709]	valid_0's l1: 505629
[14710]	valid_0's l1: 505629
[14711]	valid_0's l1: 505629
[14712]	valid_0's l1: 505628
[14713]	valid_0's l1: 505626
[14714]	valid_0's l1: 505626
[14715]	valid_0's l1: 505626
[14716]	valid_0's l1: 505629
[14717]	valid_0's l1: 505629
[14718]	valid_0's l1: 505633
[14719]	valid_0's l1: 505630
[14720]	valid_0's l1: 505634
[14721]	valid_

[14971]	valid_0's l1: 505678
[14972]	valid_0's l1: 505679
[14973]	valid_0's l1: 505675
[14974]	valid_0's l1: 505670
[14975]	valid_0's l1: 505669
[14976]	valid_0's l1: 505671
[14977]	valid_0's l1: 505669
[14978]	valid_0's l1: 505667
[14979]	valid_0's l1: 505665
[14980]	valid_0's l1: 505666
[14981]	valid_0's l1: 505666
[14982]	valid_0's l1: 505665
[14983]	valid_0's l1: 505666
[14984]	valid_0's l1: 505667
[14985]	valid_0's l1: 505667
[14986]	valid_0's l1: 505669
[14987]	valid_0's l1: 505669
[14988]	valid_0's l1: 505670
[14989]	valid_0's l1: 505670
[14990]	valid_0's l1: 505672
[14991]	valid_0's l1: 505670
[14992]	valid_0's l1: 505668
[14993]	valid_0's l1: 505667
[14994]	valid_0's l1: 505667
[14995]	valid_0's l1: 505668
[14996]	valid_0's l1: 505667
[14997]	valid_0's l1: 505668
[14998]	valid_0's l1: 505668
[14999]	valid_0's l1: 505669
[15000]	valid_0's l1: 505668
[15001]	valid_0's l1: 505670
[15002]	valid_0's l1: 505668
[15003]	valid_0's l1: 505668
[15004]	valid_0's l1: 505670
[15005]	valid_

In [15]:
prediccionLGBM = model.predict(x_test)

In [16]:
print(mean_absolute_error(y_test, prediccionLGBM))

505625.6784879495


# xgb

In [ ]:
xgb_model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, eval_metric='mae',
             gamma=0.015, importance_type='gain', learning_rate=0.025,
             max_delta_step=0, max_depth=11, min_child_weight=1, missing=None,
             n_estimators=3500, n_jobs=None, nthread=-1, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.733, verbosity=1)


#Fitteo
xgb_model.fit(x_train, y_train)
prediccion = xgb_model.predict(x_test)

In [ ]:
#Estimo MAE
print(mean_absolute_error(y_test, prediccion))

# Ensamble

In [ ]:
ensamble = (prediccion*0.5 + prediccionLGBM*0.5)

In [ ]:
#Estimo MAE
print(mean_absolute_error(y_test, ensamble))